In [47]:
from bs4 import BeautifulSoup
import requests
import re
import json
import os

root_url = r'http://ufa.hh.ru'
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 OPR/73.0.3856.344'}

In [48]:
def clean_text(string, no_list=[r'\r', r'\t', r"""<div data-qa=\"resume-block-experience-description\">""", r'</div>', r'•']):
  if not isinstance(string, str):
    string = str(string)
  return re.sub('|'.join(no_list), '', string)

In [49]:
def parse_hh_cv(cv_url):

    json = {'href': cv_url,
            'exp': [],  
            'edu': []}
    html = requests.get(cv_url, headers=HEADERS)
    html = html.text
    soup = BeautifulSoup(html, 'lxml')

    exp_all = soup.find_all('div', {'data-qa': 'resume-block-experience'})
    exp_jobs_orgs = list(map(lambda x: x.text, exp_all[0].find_all('div', {'class': 'bloko-text-emphasis'})))
    exp_jobs_orgs = list(zip(exp_jobs_orgs[::2], exp_jobs_orgs[1::2]))
    exp_info = list(map(lambda x: clean_text(x).split('\n'), exp_all[0].find_all('div', {'data-qa': 'resume-block-experience-description'})))
    exp_info = list(zip(exp_jobs_orgs, exp_info))

    for i, j in exp_info:
        base_exp = {'org': '', 'job': '', 'all_data': ''}
        base_exp['org'] = i[0]
        base_exp['job'] = i[1]
        base_exp['all_data'] = j
        json['exp'].append(base_exp)

    edu_all = soup.find_all('div', {'data-qa': 'resume-block-education'}) # bloko-text-emphasis
    edu_orgs = list(map(lambda x: x.text, edu_all[0].find_all('div', {'data-qa': 'resume-block-education-name'})))
    edu_job = list(map(lambda x: x.text, edu_all[0].find_all('div', {'data-qa': 'resume-block-education-organization'})))
    edu_all = list(zip(edu_orgs, edu_job))

    for i, j in edu_all:
        base_edu = {'org': '', 'facility': ''}
        base_edu['org'] = i
        base_edu['facility'] = j
        json['edu'].append(base_edu)

    return json

In [51]:
#parse_hh_cv(r'https://ufa.hh.ru/resume/1893ce2e00074d4e330039ed1f547a494f4a6d?hhtmFrom=resume_search_result')

In [58]:
def write_json(path=r'/content/drive/MyDrive/SberProjs/hh_cvs/hh_cvs_page_%s.json', max_page=250):

    directory, name = os.path.split(path)

    for PAGE_NUM in range(max_page+1):

        url = f'https://hh.ru/search/resume?clusters=true&no_magic=true&ored_clusters=true&search_period=30&order_by=publication_time&page={PAGE_NUM}' 

        html = requests.get(url, headers=HEADERS)
        html = html.text
        soup = BeautifulSoup(html, 'lxml')

        div_tags = soup.find_all('a', {'class': 'resume-search-item__name'})
        refs = [re.compile(r'(?<=href=)\"[^ ]+\"').findall(str(i))[0][1:-1] for i in div_tags]
        refs = list(filter(lambda x: x and 'resume' in x, refs))

        print('page №', PAGE_NUM)
        resumes = []
        for i in refs:
            try:
              dict_json = parse_hh_cv(root_url + i)
            except:
              print(root_url + i)
              continue
            resumes.append(dict_json)
        
        json.dump(resumes, open(path%PAGE_NUM, 'w', encoding='utf-8'))
        print('Saved!')

    # return resumes


In [59]:
#parse_hh_cv(r'https://ufa.hh.ru/resume/4f6a69f60001ab56990039ed1f4559647a4e4b?hhtmFrom=resume_search_result')

In [ ]:
q = write_json()

page № 0
Saved!
page № 1
Saved!
page № 2
http://ufa.hh.ru/resume/8d4f6bb900081b17c50039ed1f6b654459304f?hhtmFrom=resume_search_result
Saved!
page № 3
Saved!
page № 4
http://ufa.hh.ru/resume/9dde043a0008a275cd0039ed1f723044566b38?hhtmFrom=resume_search_result
Saved!
page № 5
http://ufa.hh.ru/resume/89013b560008a7d8930039ed1f44506761486b?hhtmFrom=resume_search_result
Saved!
page № 6
http://ufa.hh.ru/resume/d90be8ba0008b7e9a30039ed1f426779546166?hhtmFrom=resume_search_result
Saved!
page № 7
http://ufa.hh.ru/resume/fe9b92880008bcbf6d0039ed1f79467571476c?hhtmFrom=resume_search_result
Saved!
page № 8
http://ufa.hh.ru/resume/fe9b92880008bcbf6d0039ed1f79467571476c?hhtmFrom=resume_search_result
Saved!
page № 9
http://ufa.hh.ru/resume/3829c0bc00085bed4a0039ed1f34756a747171?hhtmFrom=resume_search_result
http://ufa.hh.ru/resume/7ce20c070002821a290039ed1f364a6a6e6e58?hhtmFrom=resume_search_result
http://ufa.hh.ru/resume/df84448200033621ae0039ed1f63624a373062?hhtmFrom=resume_search_result
Saved!
pag

In [53]:
url = f'https://hh.ru/search/resume?' 

html = requests.get(r'https://hh.ru/search/resume?clusters=true&no_magic=true&ored_clusters=true&search_period=30&order_by=publication_time&page=0', headers=HEADERS)
html = html.text
soup = BeautifulSoup(html, 'lxml')

div_tags = soup.find_all('a', {'class': 'resume-search-item__name'})
refs = [re.compile(r'(?<=href=)\"[^ ]+\"').findall(str(i))[0][1:-1] for i in div_tags]
refs = list(filter(lambda x: x and 'resume' in x, refs))

print(refs)

['/resume/0b3008f700042aca2a0039ed1f386246367a50?hhtmFrom=resume_search_result', '/resume/6d6499be0007cdb9e50039ed1f41336141794b?hhtmFrom=resume_search_result', '/resume/9bd6a0fe00073a57dc0039ed1f705a664d306f?hhtmFrom=resume_search_result', '/resume/b255a1f3000223bb970039ed1f6c45314c5138?hhtmFrom=resume_search_result', '/resume/8460e19100087fe05b0039ed1f414c42385138?hhtmFrom=resume_search_result', '/resume/98a137be0000d719d90039ed1f6e6f62566342?hhtmFrom=resume_search_result', '/resume/90e9da5d00020dc70e0039ed1f6b3875683349?hhtmFrom=resume_search_result', '/resume/67ae69ce00089281ea0039ed1f4a67637a5872?hhtmFrom=resume_search_result', '/resume/1908f2ea000324aec10039ed1f6c4641797245?hhtmFrom=resume_search_result', '/resume/e4b0b7610000ef66d00039ed1f306167573852?hhtmFrom=resume_search_result', '/resume/4e8b01a400013fa7500039ed1f534f7942486d?hhtmFrom=resume_search_result', '/resume/6de23ec000019c4d920039ed1f463446674c76?hhtmFrom=resume_search_result', '/resume/3d18d3ed0005d2f8580039ed1f716d

In [57]:
len(json.load(open(r'/content/drive/MyDrive/SberProjs/hh_cvs.json', 'r', encoding='utf-8')))

20

In [ ]:
parse_hh_cv()

{'edu': [{'facility': 'механико-машиностроительный, Программное обеспечение вычислительной техники и автоматизированных систем',
   'org': 'Саратовский государственный технический университет им. Ю.А. Гагарина, Саратов'}],
 'exp': [{'all_data': ['- администрирование 120 рабочих станций: диагностика программной и аппаратной части, консультирование пользователей, установка и настройка периферийного оборудования, устранение неполадок в системе Windows, Linux, Server, поддержку работы сети Интернет, АТС, резервное копирование данных',
    '- организация закупки компьютеров и оргтехники;',
    '- составление технической документации;',
    '- составление отчетов'],
   'job': 'Системный администратор',
   'org': 'государственное учреждение здравоохранения "Энгельсская городская поликлиника №1"'},
  {'all_data': ['- заключение договоров с юридическими лицами;',
    '- коммерческие предложения;',
    '- организация закупки компьютеров и оргтехники;',
    '- диагностика программной и аппаратной

In [ ]:
import json, os
from itertools import chain

q = []

for i in os.listdir(r'/content/drive/MyDrive/SberProjs/hh_cvs'):
    t = [i['href'] for i in json.load(open(os.path.join(r'/content/drive/MyDrive/SberProjs/hh_cvs', i), 'r'))]
    q.append(t)

Q = list(chain(*q))

{'edu': [{'facility': '', 'org': ''},
  {'facility': 'механико-машиностроительный, Программное обеспечение вычислительной техники и автоматизированных систем',
   'org': 'Саратовский государственный технический университет им. Ю.А. Гагарина, Саратов'}],
 'exp': [{'all_data': ['- администрирование 120 рабочих станций: диагностика программной и аппаратной части, консультирование пользователей, установка и настройка периферийного оборудования, устранение неполадок в системе Windows, Linux, Server, поддержку работы сети Интернет, АТС, резервное копирование данных',
    '- организация закупки компьютеров и оргтехники;',
    '- составление технической документации;',
    '- составление отчетов'],
   'job': 'Системный администратор',
   'org': 'государственное учреждение здравоохранения "Энгельсская городская поликлиника №1"'},
  {'all_data': ['- заключение договоров с юридическими лицами;',
    '- коммерческие предложения;',
    '- организация закупки компьютеров и оргтехники;',
    '- диагн

In [ ]:
exp_info[0]

(('ООО ГеоКИН', 'Дизайнер'),
 ["('Ферст Стрим Бизнес ', 'Помощник руководителя')"])

In [ ]:
[i.text for i in edu_orgs]

['Западно-Казахстанский Инженерно-Технологический Университет']

In [ ]:
[i.text for i in edu_job]

['Финансы и кредит, экономист']

In [ ]:
for i in exp_info:
  print(i)
  print('*'*20)



********************
•Управление производственной деятельностью коллектива 35 человек. 
•	Обеспечение выполнения производственного плана. 
•	Контроль соблюдения тех процесса. Контроль наладки и правильной эксплуатации оборудования. 
•	Заказ и контроль движения материалов и комплектующих (работа в ERP) 
•	Отчётность перед руководством. 
•	Отчётность в производственной операционной системе (ERP). 
•	Взаимодействие с вспомогательными службами

Ключевые показатели и достижения:
•	Увеличил производительность на 10% в результате устранения потерь времени на оборудовании. 
•	Внедрил "Бережливое производство" на участке по производству колёсных пар, снизив количество брака.
•	в 2017 году запустил новую производственную линию. 
•	Собрал и подготовил команду (35 человек), которая является лучшей бригадой цеха по производственным и профессиональным показателям. Средняя выработка за месяц выше на 5% чем в других сменах.
•	Обучил мастеров и бригадиров участка (8 человек) работе с системой ERP. (ра

In [ ]:
re.sub('asd', '', 'asdassdasdasdvfvsdfv')

'assdvfvsdfv'